In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from vqf import get_classical_energy, get_pauli_str, get_cost_hamiltonian

from qiskit.utils import algorithm_globals
from qiskit.algorithms.optimizers import COBYLA
from qiskit import Aer, transpile
from qiskit.algorithms import QAOA

Consider the factoring of the biprime

$$
m = p * q
$$

where, 

$$
m = \sum_{k=0}^{n_m -1} 2^k m_k \;\;\;, m_k \in \{0,1 \}
$$ 

$$
p = \sum_{k=0}^{n_p -1} 2^k p_k \;\;\;, p_k \in \{0,1 \}
$$ 

$$
q = \sum_{k=0}^{n_q -1} 2^k q_k \;\;\;, q_k \in \{0,1 \}
$$ 

We assume that 

$$
p \geq q
$$

$$
n_p = n_m
$$

$$
n_q = \left \lceil \frac{n_m}{2} \right \rceil
$$

Then,

$$
n_c = n_p + n_q - 1
$$

In [2]:
p = 7
q = 3
m = p * q

The set of clauses for the factoring problem 
$$
C_i = \sum _{j = 0} ^ i q_j p_{i-j} + \sum _{j = 0} ^ i z_{j,i} - m_i - \sum _{j = 0} ^ i 2^j z_{i, i+j} \;\;\;, 0 \leq i < n_c
$$

Hence, the classical energy is defined as

$$
E = \sum_{i=0} ^ {n_c} C_i^2
$$

In [3]:
energy = get_classical_energy(m)
energy

p_0**2*q_0**2 + p_0**2*q_1**2 + p_0**2*q_2**2 + 2*p_0*p_1*q_0*q_1 + 2*p_0*p_1*q_1*q_2 + 2*p_0*p_2*q_0*q_2 - 2*p_0*q_0 - 4*p_0*q_1*z_1_2 - 8*p_0*q_1*z_1_3 - 16*p_0*q_1*z_1_4 + 2*p_0*q_2*z_1_2 - 4*p_0*q_2*z_2_3 - 8*p_0*q_2*z_2_4 - 2*p_0*q_2 + p_1**2*q_0**2 + p_1**2*q_1**2 + p_1**2*q_2**2 + 2*p_1*p_2*q_0*q_1 + 2*p_1*p_2*q_1*q_2 + 2*p_1*p_3*q_0*q_2 - 4*p_1*q_0*z_1_2 - 8*p_1*q_0*z_1_3 - 16*p_1*q_0*z_1_4 + 2*p_1*q_1*z_1_2 - 4*p_1*q_1*z_2_3 - 8*p_1*q_1*z_2_4 - 2*p_1*q_1 + 2*p_1*q_2*z_1_3 + 2*p_1*q_2*z_2_3 - 4*p_1*q_2*z_3_4 + p_2**2*q_0**2 + p_2**2*q_1**2 + p_2**2*q_2**2 + 2*p_2*p_3*q_0*q_1 + 2*p_2*p_3*q_1*q_2 + 2*p_2*p_4*q_0*q_2 + 2*p_2*q_0*z_1_2 - 4*p_2*q_0*z_2_3 - 8*p_2*q_0*z_2_4 - 2*p_2*q_0 + 2*p_2*q_1*z_1_3 + 2*p_2*q_1*z_2_3 - 4*p_2*q_1*z_3_4 + 2*p_2*q_2*z_1_4 + 2*p_2*q_2*z_2_4 + 2*p_2*q_2*z_3_4 - 2*p_2*q_2 + p_3**2*q_0**2 + p_3**2*q_1**2 + p_3**2*q_2**2 + 2*p_3*p_4*q_0*q_1 + 2*p_3*p_4*q_1*q_2 + 2*p_3*q_0*z_1_3 + 2*p_3*q_0*z_2_3 - 4*p_3*q_0*z_3_4 + 2*p_3*q_1*z_1_4 + 2*p_3*q_1*z_2_4 + 2*p_

We apply the following classical preprocessing rules

$$
x y - 1 = 0 \Rightarrow x = y = 1,
$$

$$
x + y − 1 = 0 \Rightarrow xy = 0,
$$

$$
a − bx = 0 \Rightarrow x = 1,
$$

$$
\sum_i x_i = 0 \Rightarrow x_i = 0,
$$

$$
\sum_{i=1}^a x_i − a = 0 \Rightarrow x_i = 1.
$$

to the classical energy function to get the simplified energy function

$$
E^{'} = \sum_{i=0} ^ {n_c} C_i^{'2}
$$

In [4]:
energy = get_classical_energy(m, apply_rules=True)
energy

2*p_0*p_1*q_0*q_1 + 2*p_0*p_1*q_1*q_2 + 2*p_0*p_2*q_0*q_2 - p_0*q_0 - 4*p_0*q_1*z_1_2 - 8*p_0*q_1*z_1_3 - 16*p_0*q_1*z_1_4 + p_0*q_1 + 2*p_0*q_2*z_1_2 - 4*p_0*q_2*z_2_3 - 8*p_0*q_2*z_2_4 - p_0*q_2 + 2*p_1*p_2*q_0*q_1 + 2*p_1*p_2*q_1*q_2 + 2*p_1*p_3*q_0*q_2 - 4*p_1*q_0*z_1_2 - 8*p_1*q_0*z_1_3 - 16*p_1*q_0*z_1_4 + p_1*q_0 + 2*p_1*q_1*z_1_2 - 4*p_1*q_1*z_2_3 - 8*p_1*q_1*z_2_4 - p_1*q_1 + 2*p_1*q_2*z_1_3 + 2*p_1*q_2*z_2_3 - 4*p_1*q_2*z_3_4 + p_1*q_2 + 2*p_2*p_3*q_0*q_1 + 2*p_2*p_3*q_1*q_2 + 2*p_2*p_4*q_0*q_2 + 2*p_2*q_0*z_1_2 - 4*p_2*q_0*z_2_3 - 8*p_2*q_0*z_2_4 - p_2*q_0 + 2*p_2*q_1*z_1_3 + 2*p_2*q_1*z_2_3 - 4*p_2*q_1*z_3_4 + p_2*q_1 + 2*p_2*q_2*z_1_4 + 2*p_2*q_2*z_2_4 + 2*p_2*q_2*z_3_4 - p_2*q_2 + 2*p_3*p_4*q_0*q_1 + 2*p_3*p_4*q_1*q_2 + 2*p_3*q_0*z_1_3 + 2*p_3*q_0*z_2_3 - 4*p_3*q_0*z_3_4 + p_3*q_0 + 2*p_3*q_1*z_1_4 + 2*p_3*q_1*z_2_4 + 2*p_3*q_1*z_3_4 - p_3*q_1 + p_3*q_2 + 2*p_4*q_0*z_1_4 + 2*p_4*q_0*z_2_4 + 2*p_4*q_0*z_3_4 - p_4*q_0 + p_4*q_1 + p_4*q_2 + 16*z_1_2*z_1_3 + 32*z_1_2*z_1_4 - 

We make the ising substitution for each term of the classical energy $E^{'}$

$$
b_k \rightarrow \frac{1}{2} \left( 1 - \sigma_{b, k}^z \right)
$$

In [5]:
bits = energy.free_symbols
n_bits = len(bits)
print(f"The total number of classical bits: {n_bits}")

The total number of classical bits: 14


In [6]:
for a in range(len(energy.args)):
    term = energy.args[a]
    print(term)
    ising = get_pauli_str(term, bits)
    print(str(ising) + "\n")

3
None

3*z_1_2
1.5 * IIIIIIIIIIIIII
- 1.5 * IIIIIIIIIIIIIZ

3*z_3_4
1.5 * IIIIIIIIIIIIII
- 1.5 * IIIIIIIIIIZIII

9*z_2_3
4.5 * IIIIIIIIIIIIII
- 4.5 * IZIIIIIIIIIIII

17*z_1_3
8.5 * IIIIIIIIIIIIII
- 8.5 * IIIIIIIIIZIIII

23*z_2_4
11.5 * IIIIIIIIIIIIII
- 11.5 * IIIIIZIIIIIIII

63*z_1_4
31.5 * IIIIIIIIIIIIII
- 31.5 * IIIIZIIIIIIIII

p_0*q_1
0.25 * IIIIIIIIIIIIII
- 0.25 * IIIIIIIIIIIIZI
- 0.25 * IIIIIIIZIIIIII
+ 0.25 * IIIIIIIZIIIIZI

p_1*q_0
0.25 * IIIIIIIIIIIIII
- 0.25 * IIIIIIIIIIIZII
- 0.25 * ZIIIIIIIIIIIII
+ 0.25 * ZIIIIIIIIIIZII

p_1*q_2
0.25 * IIIIIIIIIIIIII
- 0.25 * IIIIIIIIIIIZII
- 0.25 * IIZIIIIIIIIIII
+ 0.25 * IIZIIIIIIIIZII

p_2*q_1
0.25 * IIIIIIIIIIIIII
- 0.25 * IIIIIIIIIIIIZI
- 0.25 * IIIIIIIIZIIIII
+ 0.25 * IIIIIIIIZIIIZI

p_3*q_0
0.25 * IIIIIIIIIIIIII
- 0.25 * IIIZIIIIIIIIII
- 0.25 * ZIIIIIIIIIIIII
+ 0.25 * ZIIZIIIIIIIIII

p_3*q_2
0.25 * IIIIIIIIIIIIII
- 0.25 * IIIZIIIIIIIIII
- 0.25 * IIZIIIIIIIIIII
+ 0.25 * IIZZIIIIIIIIII

p_4*q_1
0.25 * IIIIIIIIIIIIII
- 0.25 * IIIIIIIIII

By summing those terms we get the factoring cost hamiltonian 

$$
H = \sum_{i=0} ^ {n_c} C_i^{'2}
$$

In [7]:
H = get_cost_hamiltonian(m)
print(str(H))

0.125 * IIIIIIIIIIIIII
- 0.125 * IIIIIIIIIIIIZI
- 0.125 * IIIIIIZIIIIIII
+ 0.125 * IIIIIIZIIIIIZI
- 0.125 * IIIZIIIIIIIIII
+ 0.125 * IIIZIIIIIIIIZI
+ 0.125 * IIIZIIZIIIIIII
- 0.125 * IIIZIIZIIIIIZI
- 0.125 * IIZIIIIIIIIIII
+ 0.125 * IIZIIIIIIIIIZI
+ 0.125 * IIZIIIZIIIIIII
- 0.125 * IIZIIIZIIIIIZI
+ 0.125 * IIZZIIIIIIIIII
- 0.125 * IIZZIIIIIIIIZI
- 0.125 * IIZZIIZIIIIIII
+ 0.125 * IIZZIIZIIIIIZI
+ 0.125 * IIIIIIIIIIIIII
- 0.125 * IIIIIIIIIIIIZI
- 0.125 * IIIIIIZIIIIIII
+ 0.125 * IIIIIIZIIIIIZI
- 0.125 * IIIZIIIIIIIIII
+ 0.125 * IIIZIIIIIIIIZI
+ 0.125 * IIIZIIZIIIIIII
- 0.125 * IIIZIIZIIIIIZI
- 0.125 * ZIIIIIIIIIIIII
+ 0.125 * ZIIIIIIIIIIIZI
+ 0.125 * ZIIIIIZIIIIIII
- 0.125 * ZIIIIIZIIIIIZI
+ 0.125 * ZIIZIIIIIIIIII
- 0.125 * ZIIZIIIIIIIIZI
- 0.125 * ZIIZIIZIIIIIII
+ 0.125 * ZIIZIIZIIIIIZI
+ 0.125 * IIIIIIIIIIIIII
- 0.125 * IIIIIIIIZIIIII
- 0.125 * IIIIIIZIIIIIII
+ 0.125 * IIIIIIZIZIIIII
- 0.125 * IIZIIIIIIIIIII
+ 0.125 * IIZIIIIIZIIIII
+ 0.125 * IIZIIIZIIIIIII
- 0.125 * IIZIIIZIZIIIII
- 

In [8]:
algorithm_globals.random_seed = 10598
optimizer = COBYLA()
instance = Aer.get_backend("aer_simulator")
qaoa = QAOA(optimizer, quantum_instance=instance)
# result = qaoa.compute_minimum_eigenvalue(H)

### Before transpilation

In [12]:
params = np.ones(2)  # a typical QAOA ansatz has two parameters, we set them to be ones.
qc = qaoa.construct_circuit(params, H)
qc = qc[0]
# qc = qc.decompose().decompose()#.decompose()
# qc.draw('mpl')

In [16]:
n_qubits = qc.num_qubits
depth = qc.depth()
gates = qc.count_ops()
# n_cnots = gates['cx']
# n_u3 = gates['u3']
# n_gates = n_cnots + n_u3

print(
    f"The circuit parameters for factoring the biprime {m} using QAOA (no transpilation) \n"
)
print(f"The depth of the circuit: {depth}")
print(f"The total number of qubits: {n_qubits}")
print(f"The total number of gates: {str(gates)}")
# print(f"The total number of U gates: {n_u3}")
# print(f"The total number of CX gates: {n_cnots}")

The circuit parameters for factoring the biprime 21 using QAOA (no transpilation) 

The depth of the circuit: 3
The total number of qubits: 14
The total number of gates: OrderedDict([('h', 14), ('PauliEvolution', 2)])


In [17]:
for level in range(4):
    t_circ = transpile(qc, basis_gates=["cx", "u3"], optimization_level=level)
    n_qubits = t_circ.num_qubits
    depth = t_circ.depth()
    gates = t_circ.count_ops()
    n_cnots = gates["cx"]
    n_u3 = gates["u3"]
    n_gates = n_cnots + n_u3

    print(
        f"The circuit parameters for factoring the biprime {m} using QAOA  (transpilation level: {level}) \n"
    )
    print(f"The depth of the circuit: {depth}")
    print(f"The total number of qubits: {n_qubits}")
    print(f"The total number of gates: {n_gates}")
    print(f"The total number of U gates: {n_u3}")
    print(f"The total number of CX gates: {n_cnots}")
    print("--------------------------------------------")

The circuit parameters for factoring the biprime 21 using QAOA  (transpilation level: 0) 

The depth of the circuit: 774
The total number of qubits: 14
The total number of gates: 1269
The total number of U gates: 511
The total number of CX gates: 758
--------------------------------------------
The circuit parameters for factoring the biprime 21 using QAOA  (transpilation level: 1) 

The depth of the circuit: 706
The total number of qubits: 14
The total number of gates: 1183
The total number of U gates: 491
The total number of CX gates: 692
--------------------------------------------
The circuit parameters for factoring the biprime 21 using QAOA  (transpilation level: 2) 

The depth of the circuit: 706
The total number of qubits: 14
The total number of gates: 1183
The total number of U gates: 491
The total number of CX gates: 692
--------------------------------------------
The circuit parameters for factoring the biprime 21 using QAOA  (transpilation level: 3) 

The depth of the circ